<font color="red" size="6"><b>ch10.데이터 베이스 연동</b></font>

# 1절. SQLite 데이터 베이스 연결
- SQLite 데이터 베이스는 별도의 DBMS없이 SQL을 이용하여 DB 엑세스 할 수 있도록 만든 간단한 디스크 기반의 DB 제공
- C 라이브러리
- SQLite는 프로토타입을 만들 때 사용
- 프로젝트 단계 : 분석 → 설계 → 구현 → 테스트 → 고객에게 배포 → 유지보수
-               프로토타입         시제품       완제품(Oracle, MySql, Maria, ...)
-
-
- [DB browser for SQLite](https://sqlitebrowser.org/dl/)
## 1.1 SQLite broswer 설치 및 sqlite3 패키지 load

In [ ]:
import sqlite3
sqlite3.sqlite_version # sqlite3 패키지의 버전

In [ ]:
import pandas as pd
pd.__version__

In [ ]:
import numpy as np
np.__version__

## 1.2 데이터베이스 생성 및 연결
- SQLite로 DB 연결시, DB파일이 있으면 연결, DB파일이 없으면 빈 DB 파일 생성

In [ ]:
# DB 연결
conn = sqlite3.connect('data/ch10_example.db')
conn

In [ ]:
# 커서 객체 생성 커서는 SQL문 실행시키고, 결과데이터를 조회(select, 그외)하는데 사용하는 객체
cursor = conn.cursor()
cursor

In [ ]:
cursor.execute('''
    CREATE TABLE MEMBER (
        NAME TEXT,
        AGE INT,
        EMAIL TEXT
    )
''')

In [ ]:
cursor.execute('INSERT INTO MEMBER VALUES (\'홍길동\', 20, \'h@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)
sql = "INSERT INTO MEMBER VALUES ('신길동', 25, 'S@S.COM')"
cursor.execute(sql)
print('수행 결과 행수 :', cursor.rowcount)
cursor.execute('INSERT INTO MEMBER VALUES (\'신림동\', 30, \'l@h.com\')')
print('수행 결과 행수 :', cursor.rowcount)

In [ ]:
conn.commit() # 反. conn.rollback() DML문에서만 commit

In [ ]:
# SQL 전송 결과는 cursor가 가리킴
cursor.execute("SELECT * FROM MEMBER ORDER BY NAME")

In [ ]:
# insert, update, delete 문 실행결과 : cursor.rowcount
# select문 실행 결과를 받는 함수들
    ## fetchone() : 결과를 한행씩 받을 때 (튜플)
    ## fetchall() : 결과를 모두 받을 때 (튜플 list)
    ## fetchmany(n) : 결과를 n행 받을 때 (튜플 list)
print(cursor.fetchall())

In [ ]:
print(cursor.fetchall()) # 한 번 소요된 cursor 객체는 다시 fetch할 수 없음

In [ ]:
cursor.execute('SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE')
members = cursor.fetchall()
members # 튜플 리스트

In [ ]:
for member in members:
    print(member)

In [ ]:
# 한줄씩 읽기
cursor.execute("SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE")
member_list = []
while True:
    member = cursor.fetchone() # SQL문 수행 결과 한줄 가져오기
    if member is None:
        break
    #print(member)
    member_list.append( {'name':member[0], 'age':member[1],'email':member[2]})

In [ ]:
member_list

In [ ]:
# 최상위 n행 읽기
cursor.execute('SELECT NAME, AGE, EMAIL FROM MEMBER ORDER BY AGE')
for member in cursor.fetchmany(10):
    print(member)

In [ ]:
# 결과 한 줄당 객체로 받기
class Member:
    'Member 테이블의 내용을 받을 객체 타입'
    def __init__(self,name,age,email):
        self.name = name
        self.age = age
        self.email = email
    def as_dic(self): #inser문 전송시 필요
        return{'name':self.name,'age':self.age,'email':self.email}
    def __str__(self):
        return "{}\t{}\t{}".format(self.name, self.age, self.email)

def to_member(*row): # 튜플 데이터를 매개변수로 받아 Member형 객체로 return
    return Member(row[0], row[1], row[2])

In [ ]:
dbreadmember = ('홍길동', 20 ,'h@h.com')
m = to_member(*dbreadmember) # 튜플 언패킹
print(m)
print(m.as_dic())

In [ ]:
# DB 검색 결과를 객체 list로
cursor.execute('SELECT NAME, AGE, EMAIL FROM MEMBER')
member_list = [] # sql문 수행 결과를 담을 객체 list
members = cursor.fetchall() # 튜플 list
# print(members)
for member in members:
    member_list.append(to_member(*member))

In [ ]:
type(member[0]), type(member_list[0])

In [ ]:
for member in member_list:
    print(member)

In [ ]:
cursor.close() # cursor.close()는 생략 가능
conn.close()

## 1.3 SQL 구문에 파라미터 사용하기
- qmark(DB에 따라 불가한 경우가 있음)
- named(추천)

In [25]:
conn = sqlite3.connect('data/ch10_example.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN ('홍길동','신길동')")
cursor.fetchall()

[('홍길동', 20, 'h@h.com'), ('신길동', 25, 'S@S.COM')]

In [26]:
# 파라미터 사용하기 : qmark 방법 이용
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
names = (name1, name2)
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (?,?)", names)
cursor.fetchall()

KeyboardInterrupt: Interrupted by user

In [ ]:
# 파라미터 사용하기 : named 방법 이용
name1 = input('검색할 이름 > ')
cursor.execute("SELECT * FROM MEMBER WHERE NAME=:name", {'name':name})
members = cursor.fetchall()
if len(members):
    print(members)
else:
    print('해당 이름의 데이터가 없습니다')

In [27]:
# 파라미터 사용하기 : named 방법 이용
name1 = input('검색할 이름1 :')
name2 = input('검색할 이름2 :')
names = {'name1':name1,'name2':name2}
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)", {'name1':name1,'name2':name2})
cursor.execute("SELECT * FROM MEMBER WHERE NAME IN (:name1, :name2)", names)
cursor.fetchall()

검색할 이름1 :신길동
검색할 이름2 :홍길동


[('홍길동', 20, 'h@h.com'), ('신길동', 25, 'S@S.COM')]

In [ ]:
# MEMBER 테이블에 입력(사용자로부터 이름, 나이, 메일을 입력받아 insert)
try:
    name = input('입력할 이름은 : ')
    age = int(input('입력할 나이는 : '))
except ValueError:
    print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')
inputdata = {'name':name, 'age':age, 'email':email} # named 방식
inputdata2 = (name, age, email) # qmark 방식
# cursor.execute('INSERT INTO MEMBER VALUES (?,?,?)', inputdata2)
cursor.excute('INSERT INTO MEMBER VALUES (:name,:age,:email)', inputdata)
conn.commit()
if cursor.rowcount == 1:
    print('저장완료')

In [ ]:
try:
    name = input('입력할 이름은 : ')
    age = int(input('입력할 나이는 : '))
except ValueError:
    print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
    age = 18
finally:
    email = input('입력할 메일은 :')

newMember = Member(name, age, email)
# print(newMember.as_dic())
cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', newMember.as_dic())
conn.commit()
if cursor.rowcount == 1:
    print('입력 성공')

In [ ]:
# member 테이블에 데이터를 n번 입력하고
# 이름에 0을 입력할 때까지 이름, 나이, 메일을 받아 insert
# 이름에 0을 입력하면 이때까지 입력한 데이터들을 출력하고 몇명을 입력했는지도 출력

member_list=[] # 입력한 member들이 저장될 lsit변수

while True:
    try:
        name = input('입력할 이름은 : ')
        if name == '0':
            break
        age = int(input('입력할 나이는 : '))
        email = input('입력할 메일은 :')
    except ValueError:
        print('유효하지 않은 나이를 입력하신 경우 18세로 초기화')
        age = 18
        email = input('입력할 메일은 :')
       
    newMember = Member(name, age, email)
    cursor.execute('INSERT INTO MEMBER VALUES (:name, :age, :email)', newMember.as_dic())
    member_list.append(newMember)

conn.commit()

for member in member_list:
    print(member)

print('{}명 입력'.format(len(member_list)))

In [ ]:
cursor.execute('SELECT * FROM MEMBER')
cursor.fetchall()

In [ ]:
cursor.close()
conn.close()

# 2절. 오라클 데이터 베이스 연결
- cx_Oracle

In [ ]:
import cx_Oracle

In [ ]:
# conn 얻어오는 방법1
oracle_dsn = cx_Oracle.makedsn(host="localhost", port=1521, sid='xe')
print(oracle_dsn)
conn=cx_Oracle.connect('scott','tiger',dsn=oracle_dsn)
conn
# 여기서 예외가 날 경우 VC_redist.x64.exe 인스톨

In [ ]:
conn.close()

In [23]:
# conn 얻어오는 방법2
conn = cx_Oracle.connect('scott','tiger','localhost:1521/xe')
conn

NameError: name 'cx_Oracle' is not defined

In [24]:
# cursor 객체 생성
cursor = conn.cursor()
sql = "SELECT * FROM EMP"
cursor.execute(sql)
emp = cursor.fetchall()

OperationalError: no such table: EMP

In [ ]:
for e in emp:
    print(e)

In [ ]:
import pandas as pd
emp_df = pd.DataFrame(emp)
emp_df.head()

In [ ]:
# select문 수행한 필드 정보
cursor.description

In [ ]:
result = []
for description in cursor.description:
    result.append(description[0])
result

In [ ]:
[row[0] for row in cursor.description]

In [ ]:
emp_df.columns

In [ ]:
emp_df.columns = [row[0] for row in cursor.description]
emp_df

In [ ]:
# 검색할 이름을 사용자에게 받아 해당 내용을 출력
ename = input('검색할 이름은 : ')
sql = "SELECT * FROM EMP WHERE ENAME=UPPER(:name)"
cursor.execute(sql,{'name':ename})
emp = cursor.fetchall()
if len(emp):
    for e in emp:
        print(e)
else:
    print('입력하신 이름의 데이터는 없습니다')

In [ ]:
fieldnames = [description[0] for description in cursor.description]
print(fieldnames)
print(e)

In [ ]:
for idx in range(len(e)):
    print("{}:{}".format(fieldnames[idx], e[idx] if e[idx] is not None else ''))

In [ ]:
list(zip(fieldnames, e))

In [ ]:
for fieldname, data in zip(fieldnames, e):
    print("{}:{}".format(fieldname, data if data is not None else '-'))

In [ ]:
cursor.close()
conn.close()

## 3. 연습문제

In [12]:
def main():
    while True:
        print("1.입력", "2.전체 조회", "3.이름 찾기", "4.메일 삭제", "5.CSV 내보내기", "0.종료",sep=" | ")
        try:
            menu = int(input("메뉴 선택: "))
        except:
            print("유효하지 않은 값을 입력하였습니다. 다시 선택해주세요.")
        if menu==1:
            insert_member()
        elif menu==2:
            select_all()
        elif menu==3:
            select_name()
        elif menu==4:
            delete_email()
        elif menu==5:
            save_csv()
        elif menu==0:
            close_sql()
            break
if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data/data.db')
    main()

1.입력 | 2.전체 조회 | 3.이름 찾기 | 4.메일 삭제 | 5.CSV 내보내기 | 0.종료
메뉴 선택: 0


In [2]:
class Member:
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
        
    def __str__(self):
        return 'name:{}, phone:{}, email:{}, age:{}, grade:{}, etc:{}'.format(self.name,self.phone,self.email,self.age,self.grade,self.etc)
        
    def as_dict(self):
        return {'name':self.name, 'phone':self.phone, 'email':self.email,'age':self.age,'grade':self.grade,'etc':self.etc}

In [3]:
def to_member(*row): # 튜플 데이터를 매개변수로 받아 Member형 객체로 반환
    return Member(row[0],row[1],row[2],row[3],row[4],row[5])

In [4]:
# 1.입력
def insert_member():
    cursor = conn.cursor()
    sql = "INSERT INTO MEMBER VALUES (:name, :phone, :email, :age, :grade, :etc)"
    name = input("이름>")
    phone = input("전화번호>")
    email = input("이메일>")
    age = int(input("나이>"))
    grade = int(input("등급>"))
    etc = input("기타>")
    new_member=Member(name,phone,email,age,grade,etc)
    cursor.execute(sql,new_member.as_dict())

In [6]:
# 2. 전체 출력
def select_all():
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER"
    cursor.execute(sql)
    results = cursor.fetchall()
    member_list=[]
    
    for result in results:
        member_list.append(Member(*result))
        
    if len(member_list):
        for member in member_list:
            print(member)
    else:
        print("데이터가 없습니다")

In [7]:
# 3. 이름 검색
def select_name():
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER WHERE NAME = ?"
    name=input("이름>")
    cursor.execute(sql,(name,))
    
    results = cursor.fetchall()
    member_list=[]
    
    for result in results:
        member_list.append(Member(*result))
        
    if len(member_list):
        for member in member_list:
            print(member)
    else:
        print("데이터가 없습니다")

In [8]:
# 4. 메일 삭제
def delete_email():
    cursor = conn.cursor()
    sql = "DELETE FROM MEMBER WHERE EMAIL = ?"
    email = input("메일>")
    cursor.execute(sql,(email,))
    print('{}명 삭제'.format(cursor.rowcount))

In [9]:
# 5. CSV 내보내기
def save_csv():
    import csv
    
    cursor = conn.cursor()
    sql = "SELECT * FROM MEMBER"
    cursor.execute(sql)
    results = cursor.fetchall()
    member_list=[]
    
    for result in results:
        member_list.append(Member(*result).as_dict())
    
    fieldnames = [description[0] for description in cursor.description]
    
    with open('data/data.csv','w',encoding='utf-8',newline='') as f:
        dict_writer=csv.DictWriter(f,fieldnames)
        dict_writer.writerows(member_list)
    
    print("저장완료")

In [11]:
def close_sql():
    conn.commit()
    conn.close()

In [ ]:
#1 4
#2 2